### Introducción

Una vez que nuestros datos están limpios (libres de nulos problemáticos, duplicados y outliers gestionados), el siguiente paso crucial en el preprocesamiento es la **transformación de datos**. Muchos algoritmos de Machine Learning y técnicas de análisis funcionan mejor o incluso requieren que los datos de entrada estén en un formato específico.

En esta clase, nos centraremos en dos tipos principales de transformación:

1.  **Normalización y Escalado de Características Numéricas:** Ajustar la escala y distribución de las variables numéricas para que sean comparables y para mejorar el rendimiento de ciertos algoritmos.
2.  **Codificación de Variables Categóricas:** Convertir datos categóricos (texto o etiquetas) en representaciones numéricas que los algoritmos puedan procesar.

Utilizaremos **Pandas** para la manipulación de datos y **Scikit-learn (`sklearn.preprocessing`)** para las herramientas de transformación.

### 1. Configuración e Importación de Librerías

In [1]:
import pandas as pd
import numpy as np

# Importar herramientas de preprocesamiento de Scikit-learn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder

# Configuración para mostrar todas las columnas en DataFrames de Pandas (opcional)
pd.set_option('display.max_columns', None)

### 2. Carga y Preparación del Dataset (Titanic)

Continuaremos usando el dataset del Titanic. Asumiremos que ya ha pasado por una fase inicial de limpieza (manejo de nulos, etc.). Para fines de este notebook, recargaremos y aplicaremos rápidamente algunas limpiezas mínimas para tener un dataset base.

In [2]:
url_titanic = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv'
try:
    df_titanic = pd.read_csv(url_titanic)
    print("Dataset del Titanic cargado.")
except Exception as e:
    print(f"Error al cargar: {e}")
    # Crear un DataFrame vacío o con datos de ejemplo si la carga falla para que el notebook no rompa
    df_titanic = pd.DataFrame()

if not df_titanic.empty:
    # Limpieza básica (similar a la clase anterior)
    # 1. Imputar 'age' con la mediana
    df_titanic['age'].fillna(df_titanic['age'].median(), inplace=True)
    
    # 2. Imputar 'embarked' y 'embark_town' con la moda
    if 'embarked' in df_titanic.columns: 
        df_titanic['embarked'].fillna(df_titanic['embarked'].mode()[0], inplace=True)
    if 'embark_town' in df_titanic.columns: 
        df_titanic['embark_town'].fillna(df_titanic['embark_town'].mode()[0], inplace=True)
        
    # 3. Eliminar 'deck' (demasiados nulos) y 'cabin' (similar, o fuente de 'deck')
    # 'deck' es específico del dataset de seaborn, 'cabin' es más común
    if 'deck' in df_titanic.columns: 
        df_titanic.drop('deck', axis=1, inplace=True)
    if 'cabin' in df_titanic.columns: 
        df_titanic.drop('cabin', axis=1, inplace=True) # Eliminamos cabin por simplicidad aquí
        
    # 4. Eliminar filas con nulos restantes (si hay alguno en columnas críticas como 'fare')
    df_titanic.dropna(subset=['fare'], inplace=True) # Ejemplo, fare raramente es nulo
    
    # 5. Eliminar duplicados si existen (Titanic de Seaborn usualmente no tiene)
    df_titanic.drop_duplicates(inplace=True)

    print("\nDataset después de limpieza básica inicial:")
    df_titanic.info()
    display(df_titanic.head())

Dataset del Titanic cargado.

Dataset después de limpieza básica inicial:
<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     775 non-null    int64  
 1   pclass       775 non-null    int64  
 2   sex          775 non-null    object 
 3   age          775 non-null    float64
 4   sibsp        775 non-null    int64  
 5   parch        775 non-null    int64  
 6   fare         775 non-null    float64
 7   embarked     775 non-null    object 
 8   class        775 non-null    object 
 9   who          775 non-null    object 
 10  adult_male   775 non-null    bool   
 11  embark_town  775 non-null    object 
 12  alive        775 non-null    object 
 13  alone        775 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(6)
memory usage: 80.2+ KB


C:\Users\juand\AppData\Local\Temp\ipykernel_8612\862794045.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_titanic['age'].fillna(df_titanic['age'].median(), inplace=True)
C:\Users\juand\AppData\Local\Temp\ipykernel_8612\862794045.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


---

## 3. Normalización / Escalado de Características Numéricas

Muchas algoritmos de Machine Learning (especialmente los basados en distancias como KNN, SVM, o los que usan gradiente descendente como Regresión Lineal, Redes Neuronales) funcionan mejor o convergen más rápido cuando las características numéricas están en una escala similar. Si una característica tiene un rango de valores mucho mayor que otras, puede dominar el cálculo de distancias o los pasos del gradiente.

Seleccionemos las columnas numéricas que podríamos querer escalar: `age`, `fare`, `sibsp`, `parch`.

In [3]:
if not df_titanic.empty:
    cols_numericas = ['age', 'fare', 'sibsp', 'parch']
    # Asegurarse de que las columnas existan y sean numéricas
    cols_numericas_existentes = [col for col in cols_numericas if col in df_titanic.columns and pd.api.types.is_numeric_dtype(df_titanic[col])]
    
    if not cols_numericas_existentes:
        print(f"Advertencia: Ninguna de las columnas {cols_numericas} son numéricas o no existen en el DataFrame.")
        df_numerico_original = pd.DataFrame()
    else:
        print(f"Columnas numéricas seleccionadas para escalar: {cols_numericas_existentes}")
        df_numerico_original = df_titanic[cols_numericas_existentes].copy()
        display(df_numerico_original.describe())
else:
    print("DataFrame df_titanic está vacío. No se puede proceder con el escalado.")
    df_numerico_original = pd.DataFrame() # Para evitar errores más adelante

Columnas numéricas seleccionadas para escalar: ['age', 'fare', 'sibsp', 'parch']


,age,fare,sibsp,parch
count,775.000000,775.000000,775.000000,775.000000
mean,29.581187,34.878403,0.529032,0.420645
std,13.766359,52.408474,0.990326,0.840565
min,0.420000,0.000000,0.000000,0.000000
25%,21.000000,8.050000,0.000000,0.000000
50%,28.000000,15.900000,0.000000,0.000000
75%,36.000000,34.197900,1.000000,1.000000
max,80.000000,512.329200,8.000000,6.000000


### 3.1. Escalado Min-Max (Normalización)

Transforma las características escalándolas a un rango dado, comúnmente entre 0 y 1.
La fórmula es: `X_scaled = (X - X_min) / (X_max - X_min)`

In [4]:
if not df_numerico_original.empty:
    scaler_min_max = MinMaxScaler()
    
    # Fit y transform
    df_min_max_scaled_array = scaler_min_max.fit_transform(df_numerico_original)
    
    # Convertir el array resultante de nuevo a un DataFrame de Pandas
    df_min_max_scaled = pd.DataFrame(df_min_max_scaled_array, columns=df_numerico_original.columns, index=df_numerico_original.index)
    
    print("\n--- Datos después del Escalado Min-Max (rango [0, 1]) ---")
    display(df_min_max_scaled.head())
    display(df_min_max_scaled.describe()) # Media y std no serán 0 y 1, pero min y max sí.
else:
    print("No hay datos numéricos para aplicar MinMaxScaler.")


--- Datos después del Escalado Min-Max (rango [0, 1]) ---


,age,fare,sibsp,parch
0,0.271174,0.014151,0.125,0.0
1,0.472229,0.139136,0.125,0.0
2,0.321438,0.015469,0.000,0.0
3,0.434531,0.103644,0.125,0.0
4,0.434531,0.015713,0.000,0.0


,age,fare,sibsp,parch
count,775.000000,775.000000,775.000000,775.000000
mean,0.366439,0.068078,0.066129,0.070108
std,0.172988,0.102295,0.123791,0.140094
min,0.000000,0.000000,0.000000,0.000000
25%,0.258608,0.015713,0.000000,0.000000
50%,0.346569,0.031035,0.000000,0.000000
75%,0.447097,0.066750,0.125000,0.166667
max,1.000000,1.000000,1.000000,1.000000


### 3.2. Estandarización (Escalado Z-score)

Transforma las características para que tengan una media de 0 y una desviación estándar de 1.
La fórmula es: `X_scaled = (X - μ) / σ` (donde μ es la media y σ es la desviación estándar).

In [5]:
if not df_numerico_original.empty:
    scaler_standard = StandardScaler()
    
    # Fit y transform
    df_standard_scaled_array = scaler_standard.fit_transform(df_numerico_original)
    
    # Convertir de nuevo a DataFrame
    df_standard_scaled = pd.DataFrame(df_standard_scaled_array, columns=df_numerico_original.columns, index=df_numerico_original.index)
    
    print("\n--- Datos después de la Estandarización (media ~0, std ~1) ---")
    display(df_standard_scaled.head())
    display(df_standard_scaled.describe()) # Media debería ser cercana a 0, std cercana a 1.
else:
    print("No hay datos numéricos para aplicar StandardScaler.")


--- Datos después de la Estandarización (media ~0, std ~1) ---


,age,fare,sibsp,parch
0,-0.551060,-0.527515,0.475876,-0.500754
1,0.611945,0.695086,0.475876,-0.500754
2,-0.260308,-0.514627,-0.534545,-0.500754
3,0.393881,0.347909,0.475876,-0.500754
4,0.393881,-0.512240,-0.534545,-0.500754


,age,fare,sibsp,parch
count,7.750000e+02,7.750000e+02,7.750000e+02,7.750000e+02
mean,2.521281e-16,-1.134576e-16,-3.208903e-17,1.375244e-17
std,1.000646e+00,1.000646e+00,1.000646e+00,1.000646e+00
min,-2.119661e+00,-6.659405e-01,-5.345452e-01,-5.007545e-01
25%,-6.237473e-01,-5.122402e-01,-5.345452e-01,-5.007545e-01
50%,-1.149330e-01,-3.623586e-01,-5.345452e-01,-5.007545e-01
75%,4.665691e-01,-1.299298e-02,4.758756e-01,6.896894e-01
max,3.664831e+00,9.116066e+00,7.548821e+00,6.641909e+00


**¿Cuándo usar cuál?**
* **Min-Max Scaler:** Útil cuando necesitas que tus datos estén en un rango acotado (ej. [0,1]), o para algoritmos que no asumen una distribución específica (como algunos de redes neuronales con ciertas funciones de activación).
* **Standard Scaler:** A menudo preferido porque es menos sensible a outliers (aunque los outliers aún pueden influir en la media y std). Muchos algoritmos asumen que los datos están centrados alrededor de cero y tienen una varianza similar.

---

## 4. Codificación de Variables Categóricas

Los algoritmos de Machine Learning requieren entradas numéricas. Las variables categóricas (como 'sex', 'embarked', 'pclass') deben ser convertidas a números.

Seleccionemos algunas columnas categóricas del Titanic: `sex`, `embarked`, `pclass` (aunque es numérica, a menudo se trata como categórica ordinal o nominal), `who`, `adult_male`.

In [6]:
if not df_titanic.empty:
    cols_categoricas = ['sex', 'embarked', 'who', 'adult_male', 'pclass']
    # Asegurarse que las columnas existan
    cols_categoricas_existentes = [col for col in cols_categoricas if col in df_titanic.columns]
    if not cols_categoricas_existentes:
        print(f"Advertencia: Ninguna de las columnas categóricas especificadas existe.")
        df_categorico_original = pd.DataFrame()
    else:
        df_categorico_original = df_titanic[cols_categoricas_existentes].copy()
        print("Columnas categóricas seleccionadas para codificar:")
        display(df_categorico_original.head())
        print("\nTipos de datos y valores únicos:")
        for col in df_categorico_original.columns:
            print(f"- Columna '{col}': Tipo {df_categorico_original[col].dtype}, Únicos {df_categorico_original[col].nunique()}")
else:
    print("DataFrame df_titanic está vacío. No se puede proceder con la codificación.")
    df_categorico_original = pd.DataFrame() # Para evitar errores

Columnas categóricas seleccionadas para codificar:


,sex,embarked,who,adult_male,pclass
0,male,S,man,True,3
1,female,C,woman,False,1
2,female,S,woman,False,3
3,female,S,woman,False,1
4,male,S,man,True,3



Tipos de datos y valores únicos:
- Columna 'sex': Tipo object, Únicos 2
- Columna 'embarked': Tipo object, Únicos 3
- Columna 'who': Tipo object, Únicos 3
- Columna 'adult_male': Tipo bool, Únicos 2
- Columna 'pclass': Tipo int64, Únicos 3


### 4.1. Label Encoding

Asigna un entero único a cada categoría (0, 1, 2, ...). Es simple pero puede introducir una relación ordinal artificial si los datos son nominales (ej. el algoritmo podría pensar que la categoría 2 es "mayor" que la 1).
Adecuado para variables categóricas ordinales o para algoritmos basados en árboles que pueden manejar enteros categóricos.

In [7]:
if not df_categorico_original.empty and 'sex' in df_categorico_original.columns:
    df_label_encoded = df_categorico_original.copy()
    label_encoder = LabelEncoder()
    
    # Aplicar Label Encoding a la columna 'sex'
    df_label_encoded['sex_encoded'] = label_encoder.fit_transform(df_label_encoded['sex'])
    
    print("\n--- Label Encoding para 'sex' ---")
    display(df_label_encoded[['sex', 'sex_encoded']].head())
    print("Clases aprendidas por LabelEncoder para 'sex':", label_encoder.classes_)
    
    # Ejemplo con 'embarked'
    if 'embarked' in df_label_encoded.columns:
        df_label_encoded['embarked_encoded'] = label_encoder.fit_transform(df_label_encoded['embarked'])
        display(df_label_encoded[['embarked', 'embarked_encoded']].head(10))
        print("Clases aprendidas por LabelEncoder para 'embarked':", label_encoder.classes_)
else:
    print("No se puede aplicar Label Encoding porque el DataFrame categórico está vacío o 'sex' no existe.")


--- Label Encoding para 'sex' ---


,sex,sex_encoded
0,male,1
1,female,0
2,female,0
3,female,0
4,male,1


Clases aprendidas por LabelEncoder para 'sex': ['female' 'male']


,embarked,embarked_encoded
0,S,2
1,C,0
2,S,2
3,S,2
4,S,2
5,Q,1
6,S,2
7,S,2
8,S,2
9,C,0


Clases aprendidas por LabelEncoder para 'embarked': ['C' 'Q' 'S']


### 4.2. One-Hot Encoding (Variables Dummy)

Crea una nueva columna binaria (0 o 1) para cada categoría de la variable original. Evita el problema de la ordinalidad artificial del Label Encoding.
Es la técnica preferida para variables nominales en la mayoría de los algoritmos.

Pandas ofrece una función muy conveniente: `pd.get_dummies()`.

In [8]:
if not df_categorico_original.empty:
    df_one_hot_encoded = df_categorico_original.copy()
    
    # Columnas a codificar con One-Hot (nominales)
    cols_para_one_hot = [col for col in ['sex', 'embarked', 'who'] if col in df_one_hot_encoded.columns]
    
    if cols_para_one_hot:
        df_one_hot_dummies = pd.get_dummies(df_one_hot_encoded[cols_para_one_hot], prefix=cols_para_one_hot, drop_first=True)
        # drop_first=True ayuda a evitar multicolinealidad, eliminando una categoría dummy por cada variable original.
        
        print("\n--- One-Hot Encoding (Variables Dummy) con pd.get_dummies() ---")
        display(df_one_hot_dummies.head())
        
        # Unir las dummies al dataframe (opcional, podríamos hacerlo al final)
        # df_final_con_dummies = pd.concat([df_one_hot_encoded, df_one_hot_dummies], axis=1)
        # df_final_con_dummies.drop(columns=cols_para_one_hot, inplace=True) # Eliminar columnas originales
        # display(df_final_con_dummies.head())
    else:
        print("No hay columnas especificadas para One-Hot Encoding o no existen.")
else:
    print("No se puede aplicar One-Hot Encoding porque el DataFrame categórico está vacío.")


--- One-Hot Encoding (Variables Dummy) con pd.get_dummies() ---


,sex_male,embarked_Q,embarked_S,who_man,who_woman
0,True,False,True,True,False
1,False,False,False,False,True
2,False,False,True,False,True
3,False,False,True,False,True
4,True,False,True,True,False


**Usando `OneHotEncoder` de Scikit-learn (más flexible en pipelines):**
`OneHotEncoder` de `sklearn` es más potente, especialmente cuando se trabaja con conjuntos de entrenamiento y prueba por separado o dentro de `Pipelines` de Scikit-learn. Puede manejar categorías desconocidas en los datos de prueba y devuelve un array de NumPy (generalmente disperso).

In [9]:
if not df_categorico_original.empty and 'embarked' in df_categorico_original.columns:
    ohe_encoder = OneHotEncoder(sparse_output=False, drop='first') # sparse_output=False para obtener un array denso
    
    # Aplicar a 'embarked'
    embarked_ohe_array = ohe_encoder.fit_transform(df_categorico_original[['embarked']])
    
    # Crear DataFrame con las nuevas columnas
    # ohe_encoder.get_feature_names_out() da los nombres de las nuevas columnas
    df_embarked_ohe = pd.DataFrame(embarked_ohe_array, columns=ohe_encoder.get_feature_names_out(['embarked']), index=df_categorico_original.index)
    
    print("\n--- One-Hot Encoding para 'embarked' con sklearn.OneHotEncoder ---")
    display(df_embarked_ohe.head())
    print("Categorías detectadas por OneHotEncoder:", ohe_encoder.categories_)
else:
    print("No se puede aplicar OneHotEncoder porque df_categorico_original está vacío o 'embarked' no existe.")


--- One-Hot Encoding para 'embarked' con sklearn.OneHotEncoder ---


,embarked_Q,embarked_S
0,0.0,1.0
1,0.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


Categorías detectadas por OneHotEncoder: [array(['C', 'Q', 'S'], dtype=object)]


---

## 5. Aplicación de Transformaciones al DataFrame Completo

Ahora, combinemos algunas de estas técnicas para preparar una versión del `df_titanic` para un hipotético modelo.
1.  Escalaremos `age` y `fare` usando `StandardScaler`.
2.  Codificaremos `sex` y `embarked` usando `pd.get_dummies` (One-Hot Encoding).
3.  `pclass` la dejaremos como numérica por ahora, aunque a veces se codifica como categórica.

In [10]:
if not df_titanic.empty:
    df_transformado = df_titanic.copy()
    
    # 1. Escalar características numéricas ('age', 'fare')
    cols_para_escalar = ['age', 'fare']
    # Asegurarse que existan y sean numéricas
    cols_para_escalar_validas = [col for col in cols_para_escalar if col in df_transformado.columns and pd.api.types.is_numeric_dtype(df_transformado[col])]
    if cols_para_escalar_validas:
        scaler = StandardScaler()
        df_transformado[cols_para_escalar_validas] = scaler.fit_transform(df_transformado[cols_para_escalar_validas])
        print("Columnas 'age' y 'fare' estandarizadas.")
    
    # 2. Codificar características categóricas ('sex', 'embarked') con One-Hot
    cols_para_onehot = ['sex', 'embarked']
    # Asegurarse que existan
    cols_para_onehot_validas = [col for col in cols_para_onehot if col in df_transformado.columns]
    if cols_para_onehot_validas:
        df_transformado = pd.get_dummies(df_transformado, columns=cols_para_onehot_validas, drop_first=True)
        print(f"Columnas {cols_para_onehot_validas} codificadas con One-Hot (drop_first=True).")
    
    # Seleccionar solo un subconjunto de columnas para el "modelo" (y eliminar las no numéricas restantes o no útiles)
    # Por ejemplo, 'name', 'ticket', 'who', 'adult_male', 'alive', 'embark_town', 'class' podrían eliminarse o tratarse de otra forma.
    cols_a_eliminar_final = ['name', 'ticket', 'who', 'adult_male', 'alive', 'embark_town', 'class'] # 'class' es redundante con pclass
    cols_a_eliminar_existentes = [col for col in cols_a_eliminar_final if col in df_transformado.columns]
    if cols_a_eliminar_existentes:
        df_transformado.drop(columns=cols_a_eliminar_existentes, inplace=True)
        print(f"Columnas eliminadas: {cols_a_eliminar_existentes}")
        
    print("\n--- DataFrame Transformado (Primeras Filas) ---")
    display(df_transformado.head())
    df_transformado.info()
else:
    print("DataFrame Titanic está vacío. No se pueden aplicar transformaciones.")

Columnas 'age' y 'fare' estandarizadas.
Columnas ['sex', 'embarked'] codificadas con One-Hot (drop_first=True).
Columnas eliminadas: ['who', 'adult_male', 'alive', 'embark_town', 'class']

--- DataFrame Transformado (Primeras Filas) ---


,survived,pclass,age,sibsp,parch,fare,alone,sex_male,embarked_Q,embarked_S
0,0,3,-0.551060,1,0,-0.527515,False,True,False,True
1,1,1,0.611945,1,0,0.695086,False,False,False,False
2,1,3,-0.260308,0,0,-0.514627,True,False,False,True
3,1,1,0.393881,1,0,0.347909,False,False,False,True
4,0,3,0.393881,0,0,-0.512240,True,True,False,True


<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   survived    775 non-null    int64  
 1   pclass      775 non-null    int64  
 2   age         775 non-null    float64
 3   sibsp       775 non-null    int64  
 4   parch       775 non-null    int64  
 5   fare        775 non-null    float64
 6   alone       775 non-null    bool   
 7   sex_male    775 non-null    bool   
 8   embarked_Q  775 non-null    bool   
 9   embarked_S  775 non-null    bool   
dtypes: bool(4), float64(2), int64(4)
memory usage: 45.4 KB


---

## 6. Resumen

La **normalización/escalado** y la **codificación categórica** son pasos esenciales para preparar tus datos para el análisis y modelado. 
* El **escalado** asegura que las características numéricas sean comparables.
* La **codificación** convierte las variables categóricas en un formato numérico que los algoritmos puedan entender.

La elección de la técnica específica (Min-Max vs. Standard, Label vs. One-Hot) depende del tipo de datos, el algoritmo que planeas usar y las características específicas de tu dataset.

---

## 7. Ejercicios Prácticos 🧠

Utilicemos el dataset de empleados (modificado) de la clase anterior para practicar estas transformaciones.

### Ejercicio 7.1: Transformación de Datos de Empleados

**Dataset (asumamos que ya está limpio de nulos y duplicados, y outliers de 'SalarioAnual' y 'Edad' tratados con capping):**
```python
# Recrear el df_empleados (o deberías tenerlo de la clase anterior ya procesado)
datos_empleados_ej = {
    'ID_Empleado': ['E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E07', 'E08', 'E09', 'E10', 'E11'],
    'Nombre': ['Carlos', 'Ana', 'Luis', 'Sofia', 'Pedro', 'Laura', 'David', 'Maria', 'Juan', 'Desconocido', 'Elena'],
    'Edad': [34.0, 28.0, 45.0, 30.0, 35.5, 25.0, 50.0, 33.0, 48.5, 40.0, 29.0], # Edades ya limpias/tratadas
    'Departamento': ['Ventas', 'Marketing', 'TI', 'Ventas', 'TI', 'Ventas', 'RRHH', 'Ventas', 'TI', 'Marketing', 'Ventas'], # Categorías ya limpias
    'SalarioAnual': [60000.0, 75000.0, 90000.0, 62000.0, 85000.0, 76428.0, 110000.0, 58000.0, 125642.0, 72000.0, 76428.0], # Salarios ya limpios/tratados
    'FechaContratacion': pd.to_datetime(['2020-03-15', '2019-07-20', '2018-01-10', '2020-03-01', '2019-11-05', 
                          '2021-05-20', '2017-09-01', '2020-08-12', '2022-01-05', 
                          '2019-09-01', '2021-02-10'])
}
df_empleados_ej = pd.DataFrame(datos_empleados_ej)
```

**Tareas:**
1.  Crea el DataFrame `df_empleados_ej`.
2.  **Escalado Numérico:**
    * Selecciona las columnas `Edad` y `SalarioAnual`.
    * Aplica **Min-Max Scaling** a la columna `Edad` y crea una nueva columna `Edad_MinMax`.
    * Aplica **Standardization (Z-score)** a la columna `SalarioAnual` y crea una nueva columna `SalarioAnual_Std`.
3.  **Codificación Categórica:**
    * Aplica **Label Encoding** a la columna `Departamento` y crea una nueva columna `Departamento_LabelEncoded`.
    * Aplica **One-Hot Encoding** (usando `pd.get_dummies`) a la columna `Departamento` (con `drop_first=True`) y une las nuevas columnas dummy al DataFrame (puedes prefijarlas con 'Depto_').
4.  **Resultado Final:**
    * Muestra las primeras 10 filas del DataFrame `df_empleados_ej` con todas las nuevas columnas transformadas.
    * Muestra un `.describe()` de las columnas numéricas originales y las escaladas para comparar.

In [11]:
# 1. Crear el DataFrame df_empleados_ej
datos_empleados_ej = {
    'ID_Empleado': ['E01', 'E02', 'E03', 'E04', 'E05', 'E06', 'E07', 'E08', 'E09', 'E10', 'E11'],
    'Nombre': ['Carlos', 'Ana', 'Luis', 'Sofia', 'Pedro', 'Laura', 'David', 'Maria', 'Juan', 'Desconocido', 'Elena'],
    'Edad': [34.0, 28.0, 45.0, 30.0, 35.5, 25.0, 50.0, 33.0, 48.5, 40.0, 29.0], # Edades ya limpias/tratadas
    'Departamento': ['Ventas', 'Marketing', 'TI', 'Ventas', 'TI', 'Ventas', 'RRHH', 'Ventas', 'TI', 'Marketing', 'Ventas'], # Categorías ya limpias
    'SalarioAnual': [60000.0, 75000.0, 90000.0, 62000.0, 85000.0, 76428.0, 110000.0, 58000.0, 125642.0, 72000.0, 76428.0], # Salarios ya limpios/tratados
    'FechaContratacion': pd.to_datetime(['2020-03-15', '2019-07-20', '2018-01-10', '2020-03-01', '2019-11-05', 
                          '2021-05-20', '2017-09-01', '2020-08-12', '2022-01-05', 
                          '2019-09-01', '2021-02-10'])
}
df_empleados_ej = pd.DataFrame(datos_empleados_ej)
print("--- DataFrame Original de Empleados (Ejercicio) ---")
display(df_empleados_ej.head())


--- DataFrame Original de Empleados (Ejercicio) ---


,ID_Empleado,Nombre,Edad,Departamento,SalarioAnual,FechaContratacion
0,E01,Carlos,34.0,Ventas,60000.0,2020-03-15
1,E02,Ana,28.0,Marketing,75000.0,2019-07-20
2,E03,Luis,45.0,TI,90000.0,2018-01-10
3,E04,Sofia,30.0,Ventas,62000.0,2020-03-01
4,E05,Pedro,35.5,TI,85000.0,2019-11-05
